# Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ⭐️ PDF-Based RAG Q&A ChatBot Using API

In [10]:
# ╔════════════════════════════════════════════════╗
# ║ PDF Q&A RAG System with Groq Streaming LLM API ║
# ╚════════════════════════════════════════════════╝

# ── STEP 0: Install Required Libraries ──────────────────────────────────────────
!pip -q install faiss-cpu langchain-core langchain-community \
               sentence-transformers groq gradio pydantic python-dotenv pypdf PyPDF2 torch

# ── STEP 1: Imports ─────────────────────────────────────────────────────────────
import os, tempfile, traceback, torch
from pathlib import Path
from typing import List
from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from groq import Groq
import gradio as gr
from PyPDF2 import PdfReader

# ── STEP 2: Configuration ───────────────────────────────────────────────────────
GROQ_MODEL   = "meta-llama/llama-4-scout-17b-16e-instruct"
EMBED_MODEL_NAME  = "intfloat/e5-large-v2"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔋 Using Device: {DEVICE}")

EMBEDDER = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL_NAME,
    model_kwargs={"device": DEVICE}
)

# ── STEP 3: Prompt Template for LLM ─────────────────────────────────────────────
PROMPT_TEMPLATE = """
You are a highly knowledgeable assistant with full access to the contents of the provided PDF document(s).
Your task is to answer the user's questions **strictly based on the content of the PDF**.
If the answer cannot be found in the PDF, respond with: "I do not know."

Context:
{context}

Question:
{question}

Answer:
"""

# ── STEP 4: Build Vector Index from PDF Manual ──────────────────────────────────
def build_pdf_index(pdf_files: List):
    """
    Builds a searchable vector index from uploaded PDFs.
    Handles different types of file objects.
    """
    try:
        if not pdf_files:
            return None, "❌ No PDF files uploaded."

        docs = []
        log_messages = []

        for pdf in pdf_files:
            log_message = ""
            try:
                # Determine if 'pdf' is a file-like object or a file path string.
                if hasattr(pdf, "read"):
                    # Read bytes directly (works for file-like streams)
                    file_bytes = pdf.read()
                    file_name = pdf.name if hasattr(pdf, "name") else "uploaded.pdf"
                else:
                    # If pdf is a path string, open it
                    file_name = pdf
                    with open(pdf, "rb") as f:
                        file_bytes = f.read()

                log_message += f"📄 Loading: {file_name}\n"

                # Save the file bytes to a temporary file so PyPDFLoader can load it
                with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_pdf:
                    tmp_pdf.write(file_bytes)
                    tmp_pdf.flush()
                    temp_path = tmp_pdf.name
                    log_message += f"\n✅ Saved to: {temp_path}\n"

                    # Optional: Use PdfReader for debugging (e.g., getting a preview)
                    try:
                        reader = PdfReader(temp_path)
                        num_pages = len(reader.pages)
                        log_message += f"\n📋 Pages Found: {num_pages}\n"
                        if num_pages > 0:
                            preview = reader.pages[0].extract_text()
                            log_message += f"\n📝 Page 1 Preview (A few characters):\n{preview[:70]} ...\n\n"
                    except Exception as e:
                        log_message += f"❌ PyPDF2 error: {e}\n"

                    # Load the PDF using LangChain's PyPDFLoader
                    loader = PyPDFLoader(temp_path)
                    loaded_docs = loader.load()

                    if not loaded_docs:
                        log_message += f"🛑 No documents could be loaded from: {temp_path}\n"
                    else:
                        docs.extend(loaded_docs)
                        log_message += f"✅ Successfully loaded {len(loaded_docs)} pages.\n"

            except Exception as e:
                log_message += f"❌ Error loading PDF: {traceback.format_exc()}\n"

            log_messages.append(log_message)

        if not docs:
            return None, "\n".join(log_messages) + "\n❌ No documents were loaded. Check if the PDF is encrypted or corrupted."

        # Split documents into chunks
        splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
        chunks = splitter.split_documents(docs)
        log_messages.append(f"✅ Created {len(chunks)} chunks for indexing.\n")

        # Vectorize chunks
        vdb = FAISS.from_documents(chunks, EMBEDDER)

        log_messages.append(f"✅ Knowledge Base built with {len(chunks)} chunks.")
        full_log = "\n".join(log_messages)

        return vdb, full_log

    except Exception as e:
        err_msg = traceback.format_exc()
        return None, f"❌ Error during indexing:\n```\n{err_msg}\n```"

# ── STEP 5: Gradio Interface ───────────────────────────────────────────────────
with gr.Blocks(title="PDF Q&A with Groq LLM") as demo:
    gr.Markdown("## 📄 PDF Q&A Assistant – Ask Anything from Your PDF Documents")

    # Step 0: Enter API Key
    api_key_input = gr.Textbox(label="🔑 Enter your Groq API Key", type="password")

    # Step 1: Upload and Index PDF
    with gr.Row():
        pdf_files = gr.Files(label="📄 Upload Your PDF Files")
        build_button = gr.Button("🔧 Build Knowledge Base")

    # Scrollable logs
    status = gr.Textbox(label="📜 Logs and Errors", interactive=False, lines=15, max_lines=30)

    # Step 2: Ask a Question
    with gr.Row():
        question_input = gr.Textbox(label="🔍 Ask a Question")
        submit_button = gr.Button("🚀 Get Answer")

    answer_output = gr.Textbox(label="📝 Answer", interactive=False)

    # Session state for the vector store
    st_vdb = gr.State()

    # Button Event Handlers
    def build_and_store(pdfs):
        vdb, log = build_pdf_index(pdfs)
        return log, vdb

    build_button.click(fn=build_and_store, inputs=[pdf_files], outputs=[status, st_vdb])
    submit_button.click(fn=answer_question, inputs=[question_input, st_vdb, api_key_input], outputs=[answer_output])

# ── STEP 7: Launch Gradio App ──────────────────────────────────────────────────
demo.launch(share=True)


🔋 Using Device: cuda
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2d390e49d854709028.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
